Import dependancies

In [1]:
import tensorflow as tf
import os
import numpy as np
import zipfile
import matplotlib.pyplot as plt


# Make sure tenserflow version is above 2.0
try:
  %tensorflow_version 2.x
except Exvception:
  pass

Checking tensorflow version

In [2]:
tf.__version__

'2.2.0'

Download the flower dataset and unzip it

In [3]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL, fname="flower_photos.tgz", extract=True)
base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos')

228818944/228813984 [==============================] - 4s 0us/step
